# Imports 

In [ ]:
! pip install numpy

In [ ]:
!pip install pandas

In [ ]:
!pip install matplotlib

In [ ]:
!pip install torch

In [ ]:
!pip install torchvision

In [ ]:
!pip install timm

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 

import torch 
from torch import nn, optim 
from torch.utils.data import Dataset, DataLoader 

from torchvision import transforms as T

import timm

![](Untitled-design.png)

# Configurations

In [ ]:
TRAIN_CSV="train.csv"
VALID_CSV="valid.csv"

BATCH_SIZE=128
DEVICE="cpu"

MODEL_NAME="efficientnet_b0"

LR=0.001
EPOCHS=15

In [ ]:
df_train=pd.read_csv(TRAIN_CSV)
df_valid=pd.read_csv(VALID_CSV)

df_train.head()

In [ ]:
print(f"No. of examples present in df_train : {len(df_train)}")
print(f"No. of examples present in df_valid : {len(df_valid)}")
print(f"Labels are : {df_train['labels'].unique()}")

In [ ]:
idx=3100

row=df_train.iloc[idx]
image_pixels=np.array(row[:-1],dtype=np.float64)
label=row.labels

image=np.resize(image_pixels,(64,128))  #64*128=8192
plt.imshow(image)
plt.title(label);

# Declare Spec Augmentations 

![](image6.png)

In [ ]:
from spec_augment import TimeMask, FreqMask

In [ ]:
def get_train_transform():
    return T.Compose([
        TimeMask(T=15, num_masks=4),
        FreqMask(F=15, num_masks=3)  #mask width and num
    ]) 

# Create Custom Dataset 

In [ ]:
#give index and return image and label pair
class SpecDataset(Dataset):
    
    def __init__(self, df, augmentations=None):
        self.df=df
        self.augmentations=augmentations
        
        label_mapper={
            "Squiggle":0,
            "Narrowband":1,
            "Narrowbanddrd":2,
            "Noises":3
        }
        
        self.df.loc[:,"labels"]=self.df.labels.map(label_mapper) #substitute text to num
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self,idx):
        
        row=self.df.iloc[idx]
        image_pixels=np.array(row[:-1],dtype=np.float64)
        
        image=np.resize(image_pixels,(64,128,1))
        label=np.array(row.labels,dtype=np.int64)
        
        image=torch.Tensor(image).permute(2,0,1)  #(c,h,w),pytorch image passing convention
        
        if self.augmentations!= None:
            image=self.augmentations(image)
            
        return image.float(), label

In [ ]:
trainset=SpecDataset(df_train,get_train_transform())
validset=SpecDataset(df_valid)

In [ ]:
image, label = trainset[8]

plt.imshow(image.permute(0, 1, 2).squeeze())
print(label)

# Load dataset into Batches

In [ ]:
trainloader=DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True)
validloader=DataLoader(validset, batch_size=BATCH_SIZE)


In [ ]:
print(f"Total no. of batches in trainloader : {len(trainloader)}")
print(f"Total no. of batches in validloader : {len(validloader)}")

In [ ]:
for images, labels in trainloader:
    break;

print(f"One image batch shape : {images.shape}")
print(f"One label batch shape : {labels.shape}")

# Load Model

In [ ]:
#pytorch image models
class SpecModel(nn.Module):
    
    def __init__(self):
        super(SpecModel,self).__init__()
        
        self.net=timm.create_model(MODEL_NAME, num_classes=4, pretrained=True, in_chans=1) 
        
    def forward(self, images, labels=None):
        logits=self.net(images) #raw output without activation
        
        if labels != None:
            loss=nn.CrossEntropyLoss()
            return logits, loss(logits, labels)
        
        return logits
        

In [ ]:
model=SpecModel()
model;

# Create Train and Eval Function

In [ ]:
from tqdm.notebook import tqdm 
from utils import multiclass_accuracy

In [ ]:
def train_fn(model, dataloader, optimizer, current_epoch):
    model.train()
    total_loss=0.0
    total_acc=0.0
    progress_bar=tqdm(dataloader, desc="EPOCH"+"[TRAIN]"+str(current_epoch+1)+"/"+str(EPOCHS))
    
    for t, data in enumerate(progress_bar):
        images, labels=data
        images, labels=images.to(DEVICE), labels.to(DEVICE)
        
        optimizer.zero_grad()
        logits, loss=model(images, labels)
        loss.backward()
        optimizer.step()
        
        total_loss+= loss.item()
        total_acc+= multiclass_accuracy(logits, labels)
        
        temp={'loss': '%6f' %float(total_loss/(t+1)), 'acc':'%6f' %float(total_acc/(t+1))}
        
        progress_bar.set_postfix(temp)
    return total_loss/len(dataloader), total_acc/len(dataloader)
                                                                     

In [ ]:
def eval_fn(model, dataloader, current_epoch):
    model.eval()
    total_loss=0.0
    total_acc=0.0
    progress_bar=tqdm(dataloader, desc="EPOCH"+"[VALID]"+str(current_epoch+1)+"/"+str(EPOCHS))
    
    
    with torch.no_grad():
        for t, data in enumerate(progress_bar):
            images, labels=data
            images, labels=images.to(DEVICE), labels.to(DEVICE)

            logits, loss= model(images, labels)

            total_loss+= loss.item()
            total_acc+= multiclass_accuracy(logits, labels)

            temp={'loss': '%6f' %float(total_loss/(t+1)), 'acc':'%6f' %float(total_acc/(t+1))}

            progress_bar.set_postfix(temp)
    return total_loss/len(dataloader), total_acc/len(dataloader)
                                                                   

# Training Loop 

In [ ]:
def fit(model,trainloader, validloader, optimizer):
    
    best_valid_loss=np.Inf
    
    for i in range(EPOCHS):
        train_loss, train_acc=train_fn(model, trainloader, optimizer, i)
        valid_loss, valid_acc=eval_fn(model, validloader,i)
        
        if valid_loss< best_valid_loss:
            torch.save(model.state_dict(), MODEL_NAME+"-best-weights.pt")
            print("SAVED-BEST-WEIGHTS")
            
            best_valid_loss=valid_loss

In [ ]:
optimizer=torch.optim.Adam(model.parameters(),lr=LR)
fit(model, trainloader, validloader, optimizer)

# Inference 

In [ ]:
from utils import view_classify